In [38]:
import requests
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [286]:
class Wiki2pd:
    def __init__(self,**kwargs):
        self.parser = kwargs.get('parser','html.parser')
        self.decimal = kwargs.get('decimal','.')
        self.thousands = kwargs.get('thousands',',')
    # Arguments:
    # url: The URL where the Wikipedia Tables are present
    # optional:
    # table_class: string - The CSS class in which the table is nested. Default: 'wikitable'
    # tables: list - The tables, which will be downloaded. Default: [], meaning all tables
    def load_tables(self,url,**kwargs):
        self.table_class = kwargs.get('table_class','wikitable')
        #self.tables = kwargs.get('n_tables',[])
        
        soup = BeautifulSoup(requests.get(url).text, self.parser)
        tables = soup.find_all(class_=self.table_class)
        tables_df = []
        for tb in tables:
            if len(tb.find_all(class_='wikitable'))>0 or 'collapsible' in tb['class'] or len(tb.find_all('tr'))<2:
                continue
            tb = self.fix_rowspans(tb)
            
            #Remove all "sup" tags
            for tag in tb.find_all('sup'):
                tag.decompose()
                
            #Print the capture
            caption = tb.find('caption')
            if caption != None:
                print(caption.text)
            else:
                print("No Caption")
            
            #Get all rows from the table
            data = tb.find_all('tr')
            
            #Get the header from the first row
            headers_text = [x.text.strip() for x in data[0].find_all('th')]
            df = pd.DataFrame(columns=headers_text)
            
            ctr=0
            for row in data[1:]:
                #Get the row data
                data_row = dict(zip(headers_text, [x.text.strip() for x in row.find_all(['td','th'])]))
            
                if len(data_row)!=len(headers_text):
                    1+1
                    #print(data_row)
                    #print("Error on row %s: row length (%s) and header length (%s) don't match"%(ctr,len(data_row),len(headers_text)))
                else:
                    df = df.append(data_row,ignore_index=True)
            
                ctr+=1
            tables_df.append(df)
        
        #self.tables = tables_df
        return tables_df      
    
    
    def fix_rowspans(self,table):
        tmp = table.find_all('tr')
        first = tmp[0]
        allRows = tmp[1:-1]
        headers = [header.get_text() for header in first.find_all('th')]
        results = [[data.get_text() for data in row.find_all('td')] for row in allRows]
        rowspan = []

        for no, tr in enumerate(allRows):
            tmp = []
            for td_no, data in enumerate(tr.find_all('td')):
                print  data.has_key("rowspan")
                if data.has_key("rowspan"):
                    rowspan.append((no, td_no, int(data["rowspan"]), data.get_text()))


        if rowspan:
            for i in rowspan:
                # tr value of rowspan in present in 1th place in results
                for j in xrange(1, i[2]):
                    #- Add value in next tr.
                    results[i[0]+j].insert(i[1], i[3])   
        return results
    
    
    def apply_info(self,df,info):
        ll = list(df.columns)
        for col in range(len(ll)):
            if info[col].get('drop'):
                df = df.drop(columns=ll[col])
                continue
              
            if info[col].get('footnotes'):                
                df[ll[col]] = df[ll[col]].apply(lambda x: re.sub(r'\[\d+\]','',x))
                
            if info[col].get('type')=='numeric':
                new_col = df[ll[col]].apply(lambda x: x.replace(self.thousands,''))
                if info[col].get('unit')!=None:
                    new_col = new_col.apply(lambda x: x.replace(info[col].get('unit'),''))
                df[ll[col]] = pd.to_numeric(new_col,errors='coerce')
                if info[col].get('nan')!= None:
                    df[ll[col]] = df[ll[col]].fillna(info[col].get('nan'))
                    
            if info[col].get('name')!= None:
                df = df.rename(columns={ll[col]:info[col]['name']})
        return df

In [287]:
info = [{'name':'Land'},
        {'name':'Rang','type':'numeric','nan':0},
        {'name':'GRPEUR','type':'numeric',},
        {'name':'GRPUSD','type':'numeric'},
        {'name':'Comparison','drop':True}]

info2 = [{'name':'Rang'},
         {'drop':True},
         {'drop':True},
         {'drop':True},
         {'name':'GDP_Nominal','type':'numeric'},
         {'footnotes':True},
         {'name':'Population','type':'numeric'},
         {'name': 'GDP_per_Capita','type':'numeric'},
         {'drop':True}]

info3 = [{},{},{},{'type':'numeric','unit':'$'},{},{'drop':True}]

In [288]:
#url = 'https://en.wikipedia.org/wiki/List_of_German_states_by_GRP_per_capita'
#url = 'https://en.wikipedia.org/wiki/List_of_country_subdivisions_by_GDP_over_200_billion_USD'
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'
wiki = Wiki2pd()
df = wiki.load_tables(url)

#df = wiki.tables[1]
#df = wiki.apply_info(df,info3)

#df

#url = 'https://en.wikipedia.org/wiki/List_of_European_regions_by_GDP'

Highest-grossing films

Highest-grossing films as of 2019 adjusted for inflation

HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
High-grossing films by year of release

HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
HUHU
Timeline of the highest-grossing film record




## Info Object

Options:
1. Drop a column
2. Change name of a column
3. Change type of a column to numeric
    3.1 Fill NaN values
    3.2 Remove Currency icons
    
Structure:
For a table with **n** columns, you need a list with **n** dictionaries. If the dict is empty, the data will just be converted as String with out any modifications.

### 1. Dropping a column
Declare `'drop':True`, if you want to remove the column. Default = False

### 2. Change name of a column
Declare `'name':'<new_name>'`, to give the column a new name. If you leave this out, the column keeps its original name, which can be sometimes annoying, when column names are very similar or long.

Example:

`info = [
        {'name':'Land'},
        {'name':'Rang','type':'numeric','nan':0},
        {'name':'GRPEUR','type':'numeric',},
        {'name':'GRPUSD','type':'numeric'},
        {'name':'Comparison','drop':True}
        ]
`

### 3. Change type of a column to numeric

Changes the type of the Series to a numeric one, depending on the input. It removes the thousands-delimiter (normally ',', in Germany '.'). The delimiter can be selected, when creating the Wiki2pd instance. This function uses pandas `pd.to_numeric()` with `errors='coerce'`, so all errors will become NaN. Simply set `'type':'numeric'`

Example:  

`info = [
        {'name':'Land'},
        {'type':'numeric',},
        {'name':'GRPUSD','type':'numeric'}
        ]
`

####    3.1 Fill NaN values
Only works, if `'type':'numeric'` is active.
If you want to later handle NaN values yourself, you can leave this deactivated. If you just want to set NaN values to a certain value, you can use `'nan':<Value>`

Example: 

`info = [  
        {'name':'Land'},
        {'name':'Rang','type':'numeric','nan':0}  
        ]  
`

#### 3.2 Remove Units
Only works, if `'type':'numeric'` is active.
If the column contains a unit in some or all rows, `'unit':<UNIT>`removes this unit. Works by simply replacing the String with an empty String. 

Example:  

`info = [
        {},
        {'type':'numeric','unit':'€'},
        {},
        {'drop':True}
        ]
`


In [ ]:
saved_rowspans = []
for row in table.findAll("tr"):
    cells = row.findAll(["th", "td"])

    if len(saved_rowspans) == 0:
        saved_rowspans = [None for _ in cells]
        
for index, cell in enumerate(cells):
    if cell.has_key("rowspan"):
        rowspan_data = {
            'rows_left': int(cell["rowspan"]),
            'value': cell,
        }
        saved_rowspans[index] = rowspan_data
        
elif len(cells) != len(saved_rowspans):
    for index, rowspan_data in enumerate(saved_rowspans):
        if rowspan_data is not None:
            # Insert the data from previous row; decrement rows left
            cells.insert(index, rowspan_data['value'])

            if saved_rowspans[index]['rows_left'] == 1:
                saved_rowspans[index] = None
            else:
                saved_rowspans[index]['rows_left'] -= 1

In [258]:
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

tables = soup.find_all(class_="wikitable")


In [276]:
print(tables[4])

<table class="wikitable" style="margin:auto; width:100%;">
<tbody><tr>
<th style="width:5%;">Rank
</th>
<th style="width:30%;">Series
</th>
<th style="width:11%;">Total worldwide gross
</th>
<th style="width:5%;">No. of films
</th>
<th style="width:11%;">Average of films
</th>
<th style="width:38%;">Highest-grossing film
</th></tr></tbody></table>


In [1]:
xx = [1,2,3,4,5]
xx[1:-1]

[2, 3, 4]